* https://web3-ethereum-defi.readthedocs.io/tutorials/uniswap-v3-liquidity-analysis.html
* https://web3-ethereum-defi.readthedocs.io/tutorials/index.html

In [7]:
import os
import time

from web3 import HTTPProvider, Web3

from eth_defi.abi import get_contract
from eth_defi.chain import install_chain_middleware
from eth_defi.event_reader.filter import Filter
from eth_defi.event_reader.reader import read_events, LogResult
from eth_defi.event_reader.reorganisation_monitor import JSONRPCReorganisationMonitor

In [9]:
json_rpc_url = os.environ.get("JSON_RPC_POLYGON", "https://polygon-rpc.com")
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()
install_chain_middleware(web3)

# Get contracts
Pair = get_contract(web3, "sushi/UniswapV2Pair.json")

filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)

reorg_mon.load_initial_block_headers(block_count=5)

processed_events = set()

latest_block = reorg_mon.get_last_block_live()

In [10]:
while True:
    chain_reorg_resolution = reorg_mon.update_chain()
    start, end = chain_reorg_resolution.get_read_range()

    if chain_reorg_resolution.reorg_detected:
        print("Chain reorg warning")

    evt: LogResult
    for evt in read_events(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
    ):
        # How to uniquely identify EVM logs
        key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]

        # The reader may cause duplicate events as the chain tip reorganises
        if key not in processed_events:
            print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
            processed_events.add(key)
    else:
        print(".")

    if end != latest_block:
        for block_num in range(latest_block + 1, end + 1):
            block_data = reorg_mon.get_block_by_number(block_num)
            print(f"Block {block_num:,} is {block_data.block_hash}")

        latest_block = end

    time.sleep(0.5)

2024-08-01 13:33:51 eth_defi.event_reader.reorganisation_monitor figure_reorganisation_and_new_blocks(), range 60,079,539 - 60,079,581, last block we have is 60,079,542, check depth is 3


Swap at block:60,079,543 tx:0x8ccb0eb0d43830dbbc593fbf7ac896c0342d2e709201fe45aaec921a1dd5afde
Swap at block:60,079,543 tx:0x6411a6396b72fca44bb7ce1a6ba03a010b2c2e69ad10e82e5b8b173b2589e446
Swap at block:60,079,544 tx:0xd149a5600821fb25f82e262f942ebfcbb1ea1b832b1aaa6c9fea4ac6b41a690e
Swap at block:60,079,546 tx:0x41043aedb069a8045d098fbd7ac32450785921bae15bfa3a1321d0d7aec26d42
Swap at block:60,079,548 tx:0x9cb25eaa325bc80d31dc5cdc4f4c2b9859a2ac0b37d4dc9432a5b1c265efa438
Swap at block:60,079,548 tx:0x6ce59822854d1933a8f42b4d277eedc7137402e3c2e866a85f711c4f05cc65fc
Swap at block:60,079,549 tx:0x466698e1f18c32d38a5d9269fc014a68b078b90095c01dd5d56bbbcea08d8aab
Swap at block:60,079,549 tx:0x89f715a681e7eb496dd9a894d0559a65891ae73f8b076d9e875925aecd3003e5
Swap at block:60,079,549 tx:0xd3753c11ae1fb02ec731b250fbe8ca7115e4142d6934267e40e2e122c4ba5495
Swap at block:60,079,550 tx:0x19d5d7aa45604204705875271b4c4b54a269eb561e5be90cdbd72cd4870489ef
Swap at block:60,079,551 tx:0x423752a99ef04f87bcec

2024-08-01 13:34:20 eth_defi.event_reader.reorganisation_monitor figure_reorganisation_and_new_blocks(), range 60,079,578 - 60,079,594, last block we have is 60,079,581, check depth is 3


Swap at block:60,079,582 tx:0x91dbcb6bad0be5622587328920f40e83f99bf0eb4b3e34d8206ea1360a9ea26f
Swap at block:60,079,583 tx:0xd43c5e056595db71699af9cbb9e8f80920804ef918c588c176bff13638784f45
Swap at block:60,079,583 tx:0xd43c5e056595db71699af9cbb9e8f80920804ef918c588c176bff13638784f45
Swap at block:60,079,588 tx:0x5858d6b0362159d46c6cabf1b667b8083c8ccff87ad73de45a98ff9fcaa4c91f
Swap at block:60,079,588 tx:0x5bf1c4570417f4e0bc88e251415da14aeed23763ec9c16ec319d668b11241f2b
Swap at block:60,079,588 tx:0x5bf1c4570417f4e0bc88e251415da14aeed23763ec9c16ec319d668b11241f2b
Swap at block:60,079,588 tx:0x5bf1c4570417f4e0bc88e251415da14aeed23763ec9c16ec319d668b11241f2b
Swap at block:60,079,589 tx:0xb4b8e886ac8fb38f10b12c3063fd834bf59d709dc71dc5919c8055d77910f507
Swap at block:60,079,589 tx:0xb4b8e886ac8fb38f10b12c3063fd834bf59d709dc71dc5919c8055d77910f507
Swap at block:60,079,589 tx:0xb4b8e886ac8fb38f10b12c3063fd834bf59d709dc71dc5919c8055d77910f507
Swap at block:60,079,590 tx:0xc6448326e94e4c1b7316

2024-08-01 13:34:26 eth_defi.event_reader.reorganisation_monitor figure_reorganisation_and_new_blocks(), range 60,079,591 - 60,079,596, last block we have is 60,079,594, check depth is 3


...

Swap at block:60,079,659 tx:0x2740d32cd9c333503ca88a1ed9944082eb5a09d0e0e757373a9aa83d22119db7
Swap at block:60,079,659 tx:0x5be0d7c0a5d626ef3397eed513791b6f884df15ae60f5fe896a0d94d50ef5eb9
Swap at block:60,079,659 tx:0x65b4686a4ef112e686fe47feb2b76826fd84d21572c56ab94c13f2f90bc3f889
Swap at block:60,079,659 tx:0xa48a705125f53a9bf0c14a27592aae24b2965c18f305e3603a56e43cd950cb15
Swap at block:60,079,659 tx:0x9c9cb7fac81b9f700fa0574cb7ee656b64f5932adf91c349c20f152eee02bf52
Swap at block:60,079,659 tx:0x5771e04d47dbe48c6597835382fd5cd66bba87ff8ff6349ee27f666e3982cb8e
Swap at block:60,079,659 tx:0xa6aa2c0436299d268bb19dc8a92a20e208efdbe21a04fd66c4c137282365fa28
Swap at block:60,079,659 tx:0x0b88cc469bcc080fa08903c59535504af3dfc27c8adfffb2d8d563f6539b24af
Swap at block:60,079,659 tx:0x80b233306f2244a0d3ca674a008a3e8ea0c45494455aeb0a4b4053423b269143
Swap at block:60,079,659 tx:0xb9f910a777ff38bbcb6aab43f94c851ba2e2e2186e38f530e1bbee24c046dddb
Swap at block:60,079,659 tx:0xbcd53365ac156235c0d9

2024-08-01 13:36:44 eth_defi.event_reader.reorganisation_monitor figure_reorganisation_and_new_blocks(), range 60,079,657 - 60,079,662, last block we have is 60,079,660, check depth is 3


.


2024-08-01 13:36:46 eth_defi.event_reader.reorganisation_monitor figure_reorganisation_and_new_blocks(), range 60,079,657 - 60,079,663, last block we have is 60,079,660, check depth is 3


TypeError: 'NoneType' object is not subscriptable